In [ ]:
%run /content/AnnModel2.ipynb

In [ ]:
# 메인 메서드의 매개변수 추가
def binary_main(epoch_count = 10, mb_size = 10, report=1, train_ratio = 0.6, adjust_ratio = False):
    binary_load_dataset(adjust_ratio)
    init_param()
    train_and_test(epoch_count,mb_size,report,train_ratio)

In [ ]:
# 메서드 정의 

# 매개변수 
# adjust_ratio: 본 메서드에는 비율 조정에 대한 매개변수 adjust_ratio 를 할당하도록 합니다. 
# 해당 매개변수가 참(True)인 경우에만 데이터를 복사할 수 있도록 합니다. 
def binary_load_dataset(adjust_ratio:bool):
    
    # 이번에는 pulsars 데이터를 복사해야 하므로, 각 변수를 따로 저정할 필요가 있습니다. 
    # 그렇기에 두 변수를 담을 수 있는 빈 리스트를 정의합니다. 
    pulsars, stars = [],[]

    # 이전과 마찬가지로 데이터를 열고, 읽는 과정이 필요합니다. 
    with open('.../binary_classification_data.csv') as csvfile: 
        csvreader = csv.reader(csvfile)
        next(csvreader)


        '''데이터를 나눠 저장하는 과정'''
        # 현재 csvreader 변수는 마지막 열에 종속변수가 들어있기에, 
        # 해당 열의 인덱스를 기준으로 '1' 과 '0' 에 대하여 각각 빈 리스트에 할당시켜 줍니다. 
        for row in csvreader:
            
            # 펄서(1) 저장
            if row[8] == '1' : 
                pulsars.append(row)

            # 별(0) 저장
            else:
                stars.append(row)

    # 실험에 쓰일 데이터와 신경망의 입출력 크기를 전역변수로 선언합니다. 
    # pulsars, stars에 속한 데이터는 data 변수에 하나로 묶어줄 것 입니다. 
    global data, input_cnt, output_cnt
    input_cnt, output_cnt = 8,1

    # pulsars 데이터를 stars 데이터 수에 맞춰주기 위해서는 각 데이터의 개수를 파악할 필요가 있습니다.  
    star_cnt, pulsar_cnt = len(stars), len(pulsars)

    
    '''매개변수의 값이 True 인 경우 pulsars 데이터의 비율을 늘려주도록 합니다.'''
    if adjust_ratio:

        # 앞서 Star 데이터와 Pulsar 데이터는 각각 변수처리하였으나, 
        # 결국 두 데이터는 하나의 변수에 담아주어야 합니다. 
        # 그렇기에 두 데이터를 담을 수 있는 하나의 변수 data를 생성하며,
        # 해당 데이터의 크기는 star_cnt의 2배수로 저장합니다. 
        # 이유는 기존 stars 변수의 수 만큼 pulsars 변수의 수를 증가시켜 
        # 같은 수로 맞춰주기 위함입니다. 
        data = np.zeros([2 * star_cnt, 9])

        '''[stars 변수의 data 변수 할당 과정]'''
        # 할당 범위는 첫 번째 행부터 star_cnt 행 까지 
        data[0:star_cnt, :] = np.asarray(stars, dtype="float32")

        '''[pulsar 변수의 data 변수 할당 과정]'''
        # stars 변수의 수 만큼 반복하여 pulsars 데이터를 증가시켜주어야 합니다. 
        # 그렇기에 data 변수의 pulsars 데이터 할당 범위는 data[star_cnt+n] 부터 이뤄저야 하죠. 

        # 예시) star 변수와 pulsar 변수가 다음과 같이 존재합니다. 
        # star = [a,b,c,d,e]
        # pulsar = [1,2]

        # 이러한 상황에서 data 의 변수에는 다음과 같이 데이터가 할당되어야 합니다. 
        # data = [a,b,c,d,e,1,2,1,2,1]
        # 이를 고려하였을 때 pulsar 데이터의 인덱스를 순차적으로 뽑아내야 합니다. 
        
        # 파이썬의 % 연산자는 나머지를 반환하는 성질을 갖고 있으며, 
        # a % b 결괏값인 몫이 0 이하인 경우 a 를 그대로 반환합니다.  
        """ 0 % 3 = 0
            1 % 3 = 1
            2 % 3 = 2 
            3 % 3 = 0
            4 % 3 = 1 """
        # 이 점을 활용하여 pulsar 데이터를 star 데이터의 개수(star_cnt)만큼 반복해서 뽑아내줍니다. 

        for n in range(star_cnt):
            # 뽑아낸 데이터는 data[star_cnt+n] 인덱스에 할당시켜줍니다.
            data[star_cnt+n] = np.asarray(pulsars[n % pulsar_cnt], dtype='float32')


    
    # 매개변수의 값이 False 인 경우 
    # 기존 데이터를 data 변수에 그대로 할당시킵니다.
    else:
        data = np.zeros([star_cnt+pulsar_cnt,9])
        data[0:star_cnt, :] = np.asarray(stars, dtype='float32')
        data[star_cnt:,:] = np.asarray(pulsars, dtype='float32')

In [ ]:
# 메서드 재정의 
def eval_accuracy(y_hat,y_real):
		
	# 앞서 평가지표를 정의했던 코드를 그대로 가지고 옵니다. 
    est_yes = np.greater(y_hat,0)
    ans_yes = np.greater(y_real, 0.5)

    est_no = np.logical_not(est_yes) 
    ans_no = np.logical_not(ans_yes)

    tp = np.sum(np.logical_and(est_yes, ans_yes))
    tn = np.sum(np.logical_and(est_no, ans_no))
    fp = np.sum(np.logical_and(ans_no, est_yes))
    fn = np.sum(np.logical_and(ans_yes, est_no))
		
	# 안전한 나눗셈 메서드를 활용하여 줍니다. 
    accuracy = safe_div(tp+tn,tp+fp+fn+tn)
    precision = safe_div(tp,tp+fp)
    recall = safe_div(tp,tp+fn)
    f1 = 2 * safe_div(recall*precision,recall+precision)
		
	# 평가 결괏값은 4개지만, 하나의 변수로 반환하기 위해 리스트로 묶어 주었습니다. 
    return [accuracy, precision, recall, f1]

In [ ]:
# 메서드 정의
def safe_div(p, q):

    # 매개변수의 데이터 타입을 통일시켜 줍니다. 
    p, q = float(p), float(q)

    # 경우 1) 
    # accuracy = (tp+tn)/(tp+fp+fn+tn)
    # precision = tp/(tp+fp)
    # recall = (tp)/(tp+fn)
    
    # 위 세 가지 수식을 살펴보면 분모의 연산 결괏값이 0 인 경우 분자의 결괏값 또한 무조건 0 이 됩니다. 
    # 즉 분모의 값이 매우 작은 값(0 이하)이라면 안전하게 0 을 반환시켜야 됩니다. 
    if np.abs(q) < 1.0e-20:
        
        # np.sign() 메서드를 활용해 0 을 반환시켜줍니다.
        return np.sign(p)

    # 경우 2) 
    # 분모의 값이 0보다 조금이라도 크다면 일반적인 나눗셈을 수행시켜주어야 합니다. 
    return p / q

In [ ]:
# 메서드 재정의 
def train_and_test(epoch_count, mb_size, report, train_ratio):
    
    ''' train_and_test() 메서드의 기존 프로세스는 그대로 유지합니다. '''

    #1) 데이터 셔플링 - arrange_data()
    mini_batch_step_count = arrange_data(mb_size,train_ratio)
    
    #2) 테스트 데이터 분리 - get_test_data()
    test_x, test_y = get_test_data()
    
    for epoch in range(epoch_count):

        # 4개의 평가지표를 담는 빈 리스트 생성 
        losses = []

        for nth in range(mini_batch_step_count):
            # 3) 학습 데이터 분리 - get_test_data()'''
            train_x, train_y  = get_train_data(mb_size, nth)           
            
            # 4) 학습 - run_train()
            # 학습 데이터 기반의 미니배치에 따른 손실지표 cross entropy 를 반환 합니다.
            # run_train() 메서드의 두 번째 반환값 정확도는 굳이 표기하지 않기에 '_' 처리 합니다. 
            loss, _           = run_train(train_x,train_y)
            
            # 1 에폭에 따른 미니배치들의 손실값 평균을 내기 위해 값을 저장합니다.
            losses.append(loss)
            
        # 평가 및 출력은 매번 하지 않고 일정 주기마다 수행 합니다. 
        if report > 0 and (epoch+1) % report == 0:
            
            # 에폭에 따른 테스트 데이터 기반의 성능 평가 
            acc = run_test(test_x, test_y)
            # 학습 데이터 기반의 손실값 출력 / 테스트 데이터 기반의 평가지표 출력
            print("[Epoch {}] Train Loss = {:.3f} | Test Acc = {:.3f}, Precision = {:.3f}, Recall = {:.3f}, F1 = {:.3f}".\
                  format(epoch+1, np.mean(losses), acc[0],acc[1],acc[2],acc[3]))
            
    #5) 최종 테스트 - run_test()
    final_acc = run_test(test_x, test_y)
    # 평가 결과를 출력합니다. 
    print("\n","="*40, ' FINAL TEST REPORT', '='*40,"\n")
    print("► Acc = {:.3f} Precision = {:.3f}, Recall = {:.3f}, F1 = {:.3f}".\
                  format(final_acc[0],final_acc[1],final_acc[2],final_acc[3]))
